<a href="https://colab.research.google.com/github/SaeSimcheon/C_Cpp/blob/master/Extractive_QA_metric_korea_and_for_korean_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [ ]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
from datasets import load_dataset

In [ ]:
data=load_dataset('squad')

README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
gold_ex=data['train'][0]['answers']['text'][0]

In [ ]:
pred = gold_ex + '**'

In [ ]:
import re
import string

def normalize_answer(s):
    def remove_punc(text):
        korean_punctuation = '！＂＃＄％＆＇（）＊＋，－．／：；＜＝＞＠［＼］＾＿｀｛｜｝～'  # 한글 특수문자
        all_punctuation = string.punctuation + korean_punctuation
        return ''.join(ch for ch in text if ch not in all_punctuation)

    s = remove_punc(s)  # 구두점 제거
    s = re.sub(r'\s+', ' ', s).strip()  # 연속된 공백은 하나로 통합하고 양쪽 공백 제거
    return s


In [ ]:
def compute_em(pred, gold):
    return int(normalize_answer(pred) == normalize_answer(gold))

In [ ]:
compute_em(gold_ex,pred)

1

In [ ]:
!pip install konlpy

In [ ]:
from konlpy.tag import Kkma

kkma = Kkma()

# 예시 문장 분석
text = "사과를 먹었다&*^*^@@@"
print(kkma.morphs(text))  # ['사과', '를', '먹', '었', '다']

['사과', '를', '먹', '었', '다', '&', '*', '^', '*', '^', '@@@']


In [ ]:
refined_answer = normalize_answer(text)

In [ ]:
print(kkma.morphs(refined_answer))  # ['사과', '를', '먹', '었', '다']

['사과', '를', '먹', '었', '다']


In [ ]:
def em(pred,answer):
  refined_answer = normalize_answer(answer)
  refined_pred = normalize_answer(pred)

  analyzed_answer =kkma.morphs(refined_answer)
  analyzed_pred =kkma.morphs(refined_pred)

  return int(analyzed_answer == analyzed_pred)

In [ ]:
answer_text = "사과를 먹었다"
pred_text = "사과를 먹었다"
print(f"EM Score: {em(pred_text, answer_text)}")

EM Score: 1


In [ ]:
from collections import Counter
def f1score(pred,answer):
  refined_answer = normalize_answer(answer)
  refined_pred = normalize_answer(pred)

  analyzed_answer =kkma.morphs(refined_answer)
  analyzed_pred =kkma.morphs(refined_pred)

  set_answer=Counter(analyzed_answer)
  set_pred=Counter(analyzed_pred)

  c = set_answer & set_pred
  print(set_answer,set_pred,c)
  num = sum(c.values())

  precision = num / len(analyzed_answer)
  recall = num / len(analyzed_pred)
  f1 = 2 * precision * recall / (precision + recall)
  return f1

In [ ]:
answer_text = "사과를 먹고 또 사과를 먹었다"
pred_text = "사과 먹었다"
print(f"f1 Score: {f1score(pred_text, answer_text)}")

Counter({'사과': 2, '를': 2, '먹': 2, '고': 1, '또': 1, '었': 1, '다': 1}) Counter({'사과': 1, '먹': 1, '었': 1, '다': 1}) Counter({'사과': 1, '먹': 1, '었': 1, '다': 1})
f1 Score: 0.5714285714285715


In [ ]:
from google.colab import drive


drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab\ Notebooks

/content/drive/MyDrive/Colab Notebooks


In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

data = load_dataset("squad_kor_v1")

In [ ]:
import os
import requests
import json
import regex as re

In [ ]:
'''
def char_byte_map(text):

  char_to_byte = [0]*len(text)

  byte_to_char = [0]*len(text.encode('utf-8'))
  byte_index = 0
  for char_index,char in enumerate(text):

    char_to_byte[char_index] = byte_index

    length = len(char.encode('utf-8'))

    for i in range(length):
      byte_to_char[byte_index+i] = char_index

    byte_index+=length

  return char_to_byte,byte_to_char
'''

In [ ]:
def char_byte_map(text):
    char_to_byte = [0] * len(text)
    total_bytes = len(text.encode('utf-8'))
    # byte_to_char 리스트 크기를 total_bytes + 1로 설정
    byte_to_char = [0] * (total_bytes + 1)

    byte_index = 0
    for char_index, char in enumerate(text):
        char_to_byte[char_index] = byte_index

        char_bytes = char.encode('utf-8')
        length = len(char_bytes)

        for i in range(length):
            byte_to_char[byte_index + i] = char_index

        byte_index += length


    byte_to_char[total_bytes] = len(text)

    return char_to_byte, byte_to_char

In [ ]:
def bytes_to_unicode():
    bs = list(range(ord("!"), ord("~")+1))+list(range(ord("¡"), ord("¬")+1))+list(range(ord("®"), ord("ÿ")+1))
    cs = bs[:] # all integers b in bs will simply map to chr(b) in the output dict
    n = 0
    for b in range(2**8):
        if b not in bs:
            bs.append(b)
            cs.append(2**8+n)
            n += 1
    cs = [chr(n) for n in cs]
    d = dict(zip(bs, cs))
    return d

In [ ]:
def get_pairs(subwords):
  pairs = set()
  for i in range(len(subwords)-1):
    pair = (subwords[i],subwords[i+1])
    pairs.add(pair)
  return pairs

In [ ]:
def get_file(local_file, remote_file):
    """ downloads remote_file to local_file if necessary """
    if not os.path.isfile(local_file):
        print(f"downloading {remote_file} to {local_file}")
        response = requests.get(remote_file)
        open(local_file, "wb").write(response.content)

def get_encoder():
    """
    Returns an instance of the GPT BPE Encoder/Decoder
    and handles caching of "database" files.
    """
    home_dir = './'



    # load encoder.json that has the raw mappings from token -> bpe index
    encoder_local_file = os.path.join('./', 'encoder.json')
    encoder_remote_file = 'https://huggingface.co/kakaobrain/kogpt/resolve/main/tokenizer/encoder.json'
    get_file(encoder_local_file, encoder_remote_file)
    with open(encoder_local_file, 'r') as f:
        encoder = json.load(f)
    assert len(encoder) == 50257 # 256 individual byte tokens, 50,000 merged tokens, and 1 special <|endoftext|> token

    # load vocab.bpe that contains the bpe merges, i.e. the bpe tree structure
    # in the form tuples (a, b), that indicate that (a, b) is to be merged to one token ab
    vocab_local_file = os.path.join('./', 'vocab.bpe')
    vocab_remote_file = 'https://huggingface.co/kakaobrain/kogpt/resolve/main/tokenizer/vocab.bpe'
    get_file(vocab_local_file, vocab_remote_file)
    with open(vocab_local_file, 'r', encoding="utf-8") as f:
        bpe_data = f.read()
    # light postprocessing: strip the version on first line and the last line is a blank
    bpe_merges = [tuple(merge_str.split()) for merge_str in bpe_data.split('\n')[1:-1]]
    assert len(bpe_merges) == 50000 # 50,000 merged tokens

    # construct the Encoder object and return

    return encoder,bpe_merges

In [ ]:
encoder,bpe_merges=get_encoder()

In [ ]:
class Encoder :
  def __init__(self,encoder,bpe_merge_rule):

    self.byte_encoder = bytes_to_unicode()
    self.byte_decoder = {v:k for k,v in self.byte_encoder.items()}
    self.encoder = encoder
    self.decoder = {v:k for k,v in self.encoder.items()}

    self.bpe_ranks = dict(zip(bpe_merge_rule,range(len(bpe_merge_rule))))

    self.pat = re.compile(r"[\p{IsHangul}]+| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+")
    self.cache = {}

  def bpe(self,unicode_pre_token):

    word = tuple(unicode_pre_token)
    pairs = get_pairs(word)

    while pairs :

      pair = min(pairs,key = lambda pair:self.bpe_ranks.get(pair,float('inf')))
      first,second = pair
      if pair not in self.bpe_ranks :
        break
      new_word = []
      i = 0
      while i < len(word):
        if i < len(word)-1 and word[i] == first and word[i+1] == second:
          new_word.append(word[i]+word[i+1])
          i+=2
        else:
          new_word.append(word[i])
          i+=1

      word = new_word
      pairs = get_pairs(word)

    return ' '.join(word)

  def encoder_with_offsets(self,text):
    output_index = []
    offsets = []
    char_to_byte,byte_to_char = char_byte_map(text)
    for match in re.finditer(self.pat,text):
      pre_token = match.group(0)
      start_char_index_in_text = match.start()
      start_byte_index_in_text = char_to_byte[start_char_index_in_text]
      encoded_pre_token = pre_token.encode('utf-8')

      unicode_pre_token = ''.join([self.byte_encoder[b] for b in encoded_pre_token])
      subwords = self.bpe(unicode_pre_token).split(' ')

      byte_cursor = 0  # encoded_pre_token 기준
      for subword in subwords:
          # subword를 디코딩해서 실제 바이트 길이를 얻음
          subword_bytes = bytes([self.byte_decoder[c] for c in subword])
          subword_len = len(subword_bytes)

          # 전체 text에서 이 subword의 시작 byte index
          global_byte_index = start_byte_index_in_text + byte_cursor

          local_char_index = byte_to_char[global_byte_index]

          char_end_index = byte_to_char[global_byte_index+subword_len] if global_byte_index+subword_len < len(byte_to_char) else len(text)

          offsets.append((local_char_index,char_end_index))
          output_index.append(self.encoder[subword])

          byte_cursor += subword_len  # 다음 subword 위치로 이동
    return output_index,offsets

  def decode(self,index):

    joined_subwords =''.join([self.decoder[i] for i in index])

    decoded = bytearray([self.byte_decoder[c] for c in joined_subwords])


    text=decoded.decode('utf-8',errors = 'replace')

    return text

In [ ]:
enc = Encoder(encoder,bpe_merges)

In [ ]:
!pip install transformers sentencepiece